In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [2]:
torch.manual_seed(42)

In [3]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
data=pd.read_csv("/content/rice_data.csv")
data.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231.0,525.578979,229.749878,85.093788,0.928882,15617.0,0.572896,b'Cammeo'
1,14656.0,494.311005,206.020065,91.730972,0.895405,15072.0,0.615436,b'Cammeo'
2,14634.0,501.122009,214.106781,87.768288,0.912118,14954.0,0.693259,b'Cammeo'
3,13176.0,458.342987,193.337387,87.448395,0.891861,13368.0,0.640669,b'Cammeo'
4,14688.0,507.166992,211.743378,89.312454,0.906691,15262.0,0.646024,b'Cammeo'


In [5]:
data.isna().sum()

,0
Area,0
Perimeter,0
Major_Axis_Length,0
Minor_Axis_Length,0
Eccentricity,0
Convex_Area,0
Extent,0
Class,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   float64
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   float64
 6   Extent             3810 non-null   float64
 7   Class              3810 non-null   object 
dtypes: float64(7), object(1)
memory usage: 238.3+ KB


In [7]:
data['Class']=LabelEncoder().fit_transform(data['Class'])

In [8]:
X=data.drop('Class',axis=1)
y=data['Class']

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
#create a custom class
class RiceDataset(Dataset):
  def __init__(self,features,targets):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.targets=torch.tensor(targets.values,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index],self.targets[index]

In [12]:
#creating a train and test objects
train_dataset=RiceDataset(X_train,y_train)
test_dataset=RiceDataset(X_test,y_test)

In [13]:
#creating train and test data loaders
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)

In [14]:
len(train_loader)

48

In [15]:
len(test_loader)

12

In [16]:
#custom nn
class RiceClassifier(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
        nn.Linear(64,2)
    )

  def forward(self,x):
    return self.model(x)

In [17]:
#parameters
learning_rate=0.001
epochs=50

In [18]:
#instatiate the model
model=RiceClassifier(X_train.shape[1])
model=model.to(device)
#loss
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [19]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0

  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    #outputs
    outputs=model(batch_features)

    #loss
    loss=criterion(outputs,batch_labels)

    #optimizer
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss+=loss.item()

  print(f"Epoch {epoch+1}/{epochs},Loss: {total_epoch_loss/len(train_loader)}")

Epoch 1/50,Loss: 0.28122826882948476
Epoch 2/50,Loss: 0.19619469406704107
Epoch 3/50,Loss: 0.19179253559559584
Epoch 4/50,Loss: 0.1908052534175416
Epoch 5/50,Loss: 0.19097122037783265
Epoch 6/50,Loss: 0.19598745632295808
Epoch 7/50,Loss: 0.19212440323705474
Epoch 8/50,Loss: 0.18708043281609812
Epoch 9/50,Loss: 0.18622425788392624
Epoch 10/50,Loss: 0.19022074714303017
Epoch 11/50,Loss: 0.18770235870033503
Epoch 12/50,Loss: 0.1882151459964613
Epoch 13/50,Loss: 0.18911593624701103
Epoch 14/50,Loss: 0.18894467564920583
Epoch 15/50,Loss: 0.18572106212377548
Epoch 16/50,Loss: 0.18853048390398422
Epoch 17/50,Loss: 0.18795712284433344
Epoch 18/50,Loss: 0.18704603658989072
Epoch 19/50,Loss: 0.1870913840830326
Epoch 20/50,Loss: 0.18727047881111503
Epoch 21/50,Loss: 0.1849060800547401
Epoch 22/50,Loss: 0.18760928014914194
Epoch 23/50,Loss: 0.18485146900638938
Epoch 24/50,Loss: 0.1839266762447854
Epoch 25/50,Loss: 0.18775359184170762
Epoch 26/50,Loss: 0.18879068099583188
Epoch 27/50,Loss: 0.184758

In [20]:
#evaluation
model.eval()

RiceClassifier(
  (model): Sequential(
    (0): Linear(in_features=7, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [21]:
#evaluation
total=0
correct=0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    #outputs
    outputs=model(batch_features)

    #predictions
    _,predicted=torch.max(outputs,1)
    total+=batch_labels.size(0)

    correct+=(predicted==batch_labels).sum().item()

  print(f"Accuracy:{100*correct/total}")

Accuracy:92.78215223097114


In [22]:
correct=0
total=0

with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    output=model(batch_features)

    _,predicted=torch.max(output,1)
    total+=batch_labels.size(0)

    correct+=(predicted==batch_labels).sum().item()

  print(f"Accuracy:{100*correct/total}")

Accuracy:93.0774278215223
